In [1]:
import csv
import requests
from pprint import pprint
from typing import Any, Dict

In [8]:
def fetch_disasters(api_url: str, payload: Dict[str, Any]) -> Dict[str, Any]:
    """
    Fetch disasters from ReliefWeb API using a POST request.
    :param api_url: URL of the ReliefWeb API endpoint.
    :param payload: JSON payload for the POST request.
    :return: Response JSON data from the API.
    """
    headers = {
        'Content-Type': 'application/json'
    }
    
    response = requests.post(url=api_url, json=payload, headers=headers)
    response.raise_for_status()  # Raise an exception for HTTP errors
    
    return response.json()

def write_disasters_to_csv(disasters_data: Dict[str, Any], filename: str) -> None:
    """
    Write disasters data to a CSV file.
    :param disasters_data: Dictionary containing disasters data.
    :param filename: Name of the file to write to.
    """
    fields = ['ID', 'Name', 'URL', 'Status', 'Type Names', 'Country Names', 'Date Created']
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(fields)
        
        for disaster in disasters_data['data']:
            id = disaster['id']
            name = disaster['fields']['name']
            url = disaster['fields']['url']
            status = disaster['fields']['status']
            type_names = ', '.join([t['name'] for t in disaster['fields']['type']])
            country_names = ', '.join([c['name'] for c in disaster['fields']['country']])
            date_created = disaster['fields']['date']['created']
            
            writer.writerow([id, name, url, status, type_names, country_names, date_created])

if __name__ == "__main__":
    api_url = "https://api.reliefweb.int/v1/disasters"
    payload = {
        "offset": 0,
        "limit": 20,
        "filter": {
            "conditions": [
                {
                    "field": "type.id",
                    "value": ["4611", "4624", "4656"],
                    "operator": "OR"
                },
                {
                    "field": "country.id",
                    "value": "244"
                }
            ],
            "operator": "AND"
        },
        "preset": "latest",
        "profile": "list",
        "appname": "kesho_ai"
    }

    try:
        result = fetch_disasters(api_url=api_url, payload=payload)
        write_disasters_to_csv(disasters_data=result, filename='tanzania_disasters.csv')
        print("CSV file has been created successfully.")
    except requests.exceptions.RequestException as e:
        print("An error occurred:", e)


CSV file has been created successfully.
